In [228]:
import xml.etree.ElementTree as ET
import pandas as pd
import requests
import time


In [229]:
stations_df = pd.read_csv("data/fromAPI/StaDa.csv")
stations_df.head()



,Unnamed: 0,eva_nr,name,category,state,city,zipcode,street,long,lat
0,0,8000001,Aachen Hbf,2,Nordrhein-Westfalen,Aachen,52064,Bahnhofstr. 2a,6.091499,50.767800
1,1,8000406,Aachen-Rothe Erde,4,Nordrhein-Westfalen,Aachen,52066,Beverstr. 48,6.116475,50.770202
2,2,8000404,Aachen West,5,Nordrhein-Westfalen,Aachen,52072,Republikplatz 1,6.070715,50.780360
3,3,8000002,Aalen Hbf,3,Baden-Württemberg,Aalen,73430,Am Bahnhof 1,10.096271,48.841013
4,4,8000410,Abensberg,6,Bayern,Abensberg,93326,Bahnhofstr. 27,11.846620,48.819456


In [230]:
#dropping lower categories - comment out if you want all
stations_df.drop(labels=stations_df.query("category == 6").index, axis=0,inplace=True)
stations_df.drop(labels=stations_df.query("category == 7").index, axis=0,inplace=True)

In [231]:
#get relevant lists from df
eva_nrs = stations_df["eva_nr"].values
names = stations_df["name"].values
states = stations_df["state"].values
cities = stations_df["city"].values
zipcodes = stations_df["zipcode"].values
longs = stations_df["long"].values
lats = stations_df["lat"].values




In [232]:
plan_data = [] #list initializing
change_data = []

In [233]:
#API client ID and Secret
client_id = '5cf21bb577a46a3f0263677e5bee0969'
client_secret = '0229d92a983df1a6681e56ba6a390135'
#Header for request
headers={
        "DB-Api-Key": client_secret,
        "DB-Client-Id": client_id,
        "accept": "application/xml"
            }

In [234]:
for i in range(len(eva_nrs)):
    url_change = f"https://apis.deutschebahn.com/db-api-marketplace/apis/timetables/v1/fchg/{eva_nr[i]}"
    url_plan = f"https://apis.deutschebahn.com/db-api-marketplace/apis/timetables/v1/plan/{eva_nr[i]}/240619/21" 

    try:
        response_plan = requests.get(url_plan, headers=headers)
        response_change = requests.get(url_change, headers=headers)
    except:
        print(f"{i} response skipped")
        print(f"{names[i]} response skipped")
        time.sleep(0.2)
        continue

    try:
        plan_root = ET.fromstring(response_plan.content)
    except:
        print(f"{i} plan skipped")
        print(f"{names[i]} plan skipped")
        time.sleep(0.2)
        continue
    #plan_root = plan_tree.getroot()

    # Iterate over each 's' element in the XML
    for s in plan_root.findall('.//s'):
        s_id = s.get('id')
        s_eva = s.get('eva')
        # Extract 'ar' element data
        ar = s.find('ar')
        if ar is not None:
            ar_pt = ar.get('pt')#planned arrival
            ar_l = ar.get('l')
            ar_ppth = ar.get('ppth')
        else:
            ar_ct = ar_l = None
        # Extract 'dp' element data
        dp = s.find('dp')
        if dp is not None:
            dp_pt = dp.get('pt')
            dp_l = dp.get('l')
        else:
            dp_ct = dp_l = None  
        # Append the extracted data to the list
        plan_data.append([s_id, ar_pt, dp_pt, dp_l, ar_ppth, eva_nrs[i], names[i], states[i], cities[i], zipcodes[i], longs[i], lats[i]])

    try:
        change_root = ET.fromstring(response_change.content)
    #change_root = change_tree.getroot()
    except:
        print(f"{i} change skipped")
        print(f"{names[i]} change skipped")
        time.sleep(0.2)
        continue
    # Iterate over each 's' element in the XML
    for s in change_root.findall('.//s'):
        s_id = s.get('id')
        s_eva = s.get('eva')
        # Extract 'ar' element data
        ar = s.find('ar')
        if ar is not None:
            ar_pt = ar.get('ct')#planned arrival
            ar_l = ar.get('l')
        else:
            ar_ct = ar_l = None
        
        # Extract 'dp' element data
        dp = s.find('dp')
        if dp is not None:
            dp_pt = dp.get('ct')
            dp_l = dp.get('l')
        else:
            dp_ct = dp_l = None
        # Append the extracted data to the list
        change_data.append([s_id, ar_pt, dp_pt,])
        
    print(f"{i} of {len(eva_nrs)}")
    print(names[i])
    time.sleep(0.1)



columns = ['ID', 'arrival', "departure", "train", "path",'eva_nr', "name", "state", "city", "zip", "long", "lat"]
plan_df = pd.DataFrame(plan_data, columns=columns)
plan_df['arrival'] = pd.to_datetime(plan_df['arrival'], format='%y%m%d%H%M')
plan_df['departure'] = pd.to_datetime(plan_df['departure'], format='%y%m%d%H%M')

columns = ['ID', 'arrival', "departure", ]
change_df = pd.DataFrame(change_data, columns=columns)
change_df['arrival'] = pd.to_datetime(change_df['arrival'], format='%y%m%d%H%M')
change_df['departure'] = pd.to_datetime(change_df['departure'], format='%y%m%d%H%M')
       

0 of 2001
Aachen Hbf
1 of 2001
Aachen-Rothe Erde
2 of 2001
Aachen West
3 of 2001
Aalen Hbf
4 of 2001
Achern
5 of 2001
Achim
6 of 2001
Ahaus
7 of 2001
Ahlen (Westf)
8 of 2001
Ahlten (Han)
9 of 2001
Ahrensburg
10 of 2001
Ahrensfelde
11 of 2001
Aichach
12 of 2001
Albrechtshof
13 of 2001
Albstadt-Ebingen
14 of 2001
Alexanderplatz
15 of 2001
Alfeld (Leine)
16 of 2001
Allensbach
17 of 2001
Alsfeld (Oberhess)
18 of 2001
Alsheim
19 of 2001
Altbach
20 of 2001
Alte Wöhr
21 of 2001
Altena (Westf)
22 of 2001
Altenbeken
23 of 2001
Kurort Altenberg (Erzgeb)
24 of 2001
Altenburg
25 of 2001
Lennestadt-Altenhundem
26 of 2001
Altglienicke
27 of 2001
Althegnenberg
28 of 2001
Alzey
29 of 2001
Amberg
30 of 2001
Ampfing
31 of 2001
Andernach
32 of 2001
Anderten-Misburg
33 of 2001
Angermünde
34 of 2001
Anklam
35 of 2001
Annweiler am Trifels
36 of 2001
Anrath
37 of 2001
Ansbach
38 of 2001
Apolda
39 of 2001
Nottuln-Appelhülsen
40 of 2001
Appenweier
41 of 2001
Armsheim
42 of 2001
Arnsberg (Westf)
43 of 2001
Arns

In [235]:
plan_data = [] #emtpy lists for next iteration (for shedule script)
change_data = []

In [236]:
#merging the changes data into the planned data
delay_df = pd.merge(plan_df, change_df, how='left', on="ID", suffixes=('_plan', '_change'))
delay_df['depature_delay_m'] = delay_df['departure_change'] - delay_df['departure_plan']
delay_df['arrival_delay_m'] = delay_df['arrival_change'] - delay_df['arrival_plan']

In [237]:
#sorting columns
delay_df = delay_df[['ID', 'train', 'path', 'eva_nr',
       'name', 'state', 'city', 'zip', 'long', 'lat', 'arrival_plan', 'departure_plan', 'arrival_change',
       'departure_change',  'arrival_delay_m', 'depature_delay_m',]]

In [238]:
delay_df["depature_delay_m"] = delay_df["depature_delay_m"].dt.total_seconds()/60
delay_df["depature_delay_m"] = delay_df["depature_delay_m"].fillna(value=0)
delay_df["depature_delay_m"] = delay_df["depature_delay_m"].astype(int)
delay_df["arrival_delay_m"] = delay_df["arrival_delay_m"].dt.total_seconds()/60
delay_df["arrival_delay_m"] = delay_df["arrival_delay_m"].fillna(value=0)
delay_df["arrival_delay_m"] = delay_df["arrival_delay_m"].astype(int)
delay_df["eva_nr"] = delay_df["eva_nr"].astype(int)

In [239]:
delay_df.head()

,ID,train,path,eva_nr,name,state,city,zip,long,lat,arrival_plan,departure_plan,arrival_change,departure_change,arrival_delay_m,depature_delay_m
0,2142685495596717780-2406192117-1,18,Bergisch Gladbach|Duckterath|Köln-Dellbrück|Kö...,8000001,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.7678,2024-06-19 13:00:00,2024-06-19 21:17:00,NaT,NaT,0,0
1,-2730674727498638193-2406192034-9,20,Stolberg-Rathaus|Stolberg-Mühlener Bahnhof|Sto...,8000001,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.7678,2024-06-19 21:01:00,2024-06-19 21:02:00,2024-06-19 21:01:00,2024-06-19 21:02:00,0,0
2,-3561454673811003901-2406192137-1,29,Stolberg-Rathaus|Stolberg-Mühlener Bahnhof|Sto...,8000001,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.7678,2024-06-19 21:01:00,2024-06-19 21:37:00,NaT,NaT,0,0
3,-154666799239067997-2406192104-9,20,Stolberg-Rathaus|Stolberg-Mühlener Bahnhof|Sto...,8000001,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.7678,2024-06-19 21:31:00,2024-06-19 21:32:00,2024-06-19 21:30:00,2024-06-19 21:32:00,-1,0
4,2192262199334435396-2406192031-15,None,Heinsberg(Rheinl)|Heinsberg Kreishaus|Heinsber...,8000001,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.7678,2024-06-19 21:31:00,2024-06-19 21:32:00,2024-06-19 21:30:00,NaT,-1,0


In [240]:
delay_df.to_csv(f"data/fromAPI/delay_without6_7.csv")

In [241]:
delay_df.shape

(8590, 16)